# Compare data saving and loading performance of QCoDeS SQLite backend vs HDF5 (h5py) and numpy npy files

This notebook measures time it takes to save and load measurement data using qcodes dataset versus other ways of storing data, hdf5 and numpy npy files. The reason for such a study is that qcodes users should not be limited in the performance of the their experiments by the performance of data saving (and loading).

HDF5 and numpy npy storage solutions are not only widely used in the scientific community, but are also known for their efficiency.

### Imports

In [1]:
import time
import os
from tempfile import TemporaryFile
import timeit

import numpy
import h5py
from git import Repo
from IPython.core.magics.execution import TimeitResult

import qcodes
from qcodes import (
    initialise_or_create_database_at, load_or_create_experiment, 
    Measurement, Parameter,
    load_by_id
)
from qcodes.dataset.data_export import get_data_by_id

### Relevant environment information

In [2]:
qcodes.version.__version__

'0.1.11'

In [3]:
# in case the qcodes is installed from local git repository
qcodes_repo_path = os.sep.join(qcodes.__path__[0].split(os.sep)[:-1])
qcodes_repo = Repo(qcodes_repo_path)
print(qcodes_repo.head.commit)

8979f3bed18ff3c6d9657977efed0ce3744135c5


In [4]:
print(h5py.version.info)

Summary of the h5py configuration
---------------------------------

h5py    2.8.0
HDF5    1.10.2
Python  3.6.6 |Anaconda, Inc.| (default, Jun 28 2018, 11:27:44) [MSC v.1900 64 bit (AMD64)]
sys.platform    win32
sys.maxsize     9223372036854775807
numpy   1.15.2



# Simulated measurement

For this study, we are going to take the case of sweeping 2 independent parameters (s1, s2) and measuring 2 dependent parameters (magnitude and phase). For simplicity, the number of datapoints per parameter is the same, and it is set in a variable. We are going to use the same generator function throughout the study for generating dummy data that we will be saving and loading.

In [5]:
# number of data points per parameter
n_pts_per_param = 20

In [6]:
def produce_measurement_data(n_pts_per_param):
    """
    This iterator represents the code that obtains
    measurement data. For the sake of example, it
    just returns random dummy data: 4 parameters/dimensions, 
    `n_pts_per_param` per each dimension (which becomes
    `n_pts_per_param**4` data points in total).
    
    Args:
        n_pts_per_param
            number of points per each parameter/dimension
    
    Returns:
        tuple of values of the 4 dimensions obtained
        at a single "measurement" iteration
    """
    for s1_val in range(n_pts_per_param):
        for s2_val in range(n_pts_per_param):
            magn_vals, phas_vals = np.meshgrid(
                np.random.rand(n_pts_per_param),
                np.random.rand(n_pts_per_param),
            )
            magn_vals = np.reshape(magn_vals, -1)
            phas_vals = np.reshape(phas_vals, -1)
            
            yield s1_val, s2_val, magn_vals, phas_vals

# Test routines

Now lets define all the test routines for saving and loading data for testing the performance of different backends.

## Base class

???

we'll be using custom timeit-like thingy, because timeit does not allow the flexibility of placing the moments in the code where we want to record the time stamps.

In [87]:
def time_it(number=None,
            repeat=timeit.default_repeat):
    def time_sut(sut):
        def wrapper(*args, **kwargs):
            
            t = timeit.Timer()

            def inner(_it, _timer):
                total_time = 0
                
                for _ in _it:
                    returns = sut(*args, **kwargs)
                    
                    if isinstance(returns, tuple):
                        returns = returns[0]
                        
                    total_time += returns
                
                return total_time
            
            t.inner = inner

            try:
                if number is None:
                    number_, _ = t.autorange()
                else:
                    number_ = number
                
                all_runs = t.repeat(repeat, number_)
            except:
                t.print_exc()
                raise

            best = min(all_runs) / number_
            worst = max(all_runs) / number_
            print(TimeitResult(number_, repeat, best, worst, all_runs, 0, 3))
        return wrapper
    return time_sut

## QCoDeS dataset

First, we need to initialize a database file.

In [7]:
# initialize the database file for qcodes dataset

temp_db_file = TemporaryFile(suffix='.db')
temp_db_file.close()

initialise_or_create_database_at(temp_db_file.name)

load_or_create_experiment('save_load_speed_study', 'sqlite3_from_qcodes')

save_load_speed_study#sqlite3_from_qcodes#1@C:\Users\a-miasta\AppData\Local\Temp\tmptuva4aob.db
-----------------------------------------------------------------------------------------------

Next, we define a convenient function that performs all the usual steps that are necessary for a qcodes measurement and data saving. This function's main purpose is to measure the time it takes to save the data.

In [8]:
# @time_it(number=4, repeat=2)
def save_to_sqlite(data_generator, paramtype='numeric'):
    """
    Use qcodes dataset with its sqlite backend to save dummy
    data, and measure the time this takes. The data that is being
    saved is 2 dependent and 2 independent parameters. The data
    for the measurement is generated by the `data_generator`
    iterator function.
    
    Args:
        data_generator
            iterator that returns dummy data for 4 parameters
        paramtype
            controls the way data the 2 dependent parameters are stored
            in the sqlite database,
            see `Measurement.register_parameter` for more information
            (useful values in the context of this notebook are 'numeric'
            and 'array')
    
    Returns:
        saving_time
            measured time it took to save the data, in seconds
        dataset
            the qcodes dataset object where the data was saved to;
            it is useful for accessing the data and measuring the
            time it takes to load it
    """
    # define parameters
    s1 = Parameter('s1', label='Setting 1', unit='V', get_cmd=None, set_cmd=None)
    s2 = Parameter('s2', label='Setting 2', unit='V', get_cmd=None, set_cmd=None)
    magn = Parameter('magn', label='Magnitude', unit='V', get_cmd=None, set_cmd=None)
    phas = Parameter('phas', label='Phase', unit='deg', get_cmd=None, set_cmd=None)
    
    meas = Measurement()
    
    # register parameters in the measurement object
    meas.register_parameter(s1)
    meas.register_parameter(s2)
    meas.register_parameter(magn, setpoints=(s1, s2), paramtype=paramtype)
    meas.register_parameter(phas, setpoints=(s1, s2), paramtype=paramtype)
    
    # set the write period to a large value, so that actual writing
    # to the database happens at the end of the "measurement"
    meas.write_period = 10
    
    # perform the measurement
    with meas.run() as datasaver:
        t0 = time.perf_counter()  # <-----

        for s1_val, s2_val, magn_vals, phas_vals \
                in data_generator:
            
            datasaver.add_result((s1, s1_val), 
                                 (s2, s2_val),
                                 (magn, magn_vals),
                                 (phas, phas_vals))
    
    t1 = time.perf_counter()  # <-----
    saving_time = t1 - t0
    
    dataset = datasaver.dataset
    
    return saving_time, dataset

## Via QCoDeS DataSet

In [9]:
number = 3
repeat = 2

t = timeit.Timer()

def inner(_it, _timer):
    total_time = 0

    for _ in _it:
        returns = save_to_sqlite(
            produce_measurement_data(n_pts_per_param),
            paramtype='numeric')

        if isinstance(returns, tuple):
            returns = returns[0]

        total_time += returns

    return total_time

t.inner = inner

try:
    if number is None:
        number_, _ = t.autorange()
    else:
        number_ = number

    all_runs = t.repeat(repeat, number_)
except:
    t.print_exc()
    raise

best = min(all_runs) / number_
worst = max(all_runs) / number_
print(TimeitResult(number_, repeat, best, worst, all_runs, 0, 3))

Starting experimental run with id: 1
Starting experimental run with id: 2
Starting experimental run with id: 3
Starting experimental run with id: 4
Starting experimental run with id: 5
Starting experimental run with id: 6
3.92 s ± 74.4 ms per loop (mean ± std. dev. of 2 runs, 3 loops each)


### with 'numeric' type

In [89]:
save_to_sqlite(
    produce_measurement_data(n_pts_per_param),
    paramtype='numeric')

Starting experimental run with id: 68
inner: None: (3.775037899999006, results #68@C:\Users\a-miasta\AppData\Local\Temp\tmpwy9lzac8.db
---------------------------------------------------------------
s1 - numeric
s2 - numeric
magn - numeric
phas - numeric)
Starting experimental run with id: 69
inner: None: (0.018814400000337628, results #69@C:\Users\a-miasta\AppData\Local\Temp\tmpwy9lzac8.db
---------------------------------------------------------------
s1 - numeric
s2 - numeric
magn - numeric
phas - numeric)
Starting experimental run with id: 70
inner: None: (0.016228399999818066, results #70@C:\Users\a-miasta\AppData\Local\Temp\tmpwy9lzac8.db
---------------------------------------------------------------
s1 - numeric
s2 - numeric
magn - numeric
phas - numeric)
Starting experimental run with id: 71
inner: None: (0.017815300001529977, results #71@C:\Users\a-miasta\AppData\Local\Temp\tmpwy9lzac8.db
---------------------------------------------------------------
s1 - numeric
s2 - numeri

In [ ]:
print("Data saving to dataset with 'numeric' paramtype took:")

    

In [11]:
%%timeit

# returns list of rows, as in sqlite (each row is a list where every item corresponds to the column)
# data = dataset.get_data(*datasaver.dataset.parameters.split(','))
#
# another way is to use get_values and obtain values of each parameter one by one;
# but get_data_by_id uses it already, so let's not repeat it
# data = dataset.get_values

data = dataset.get_data(*dataset.parameters.split(','))

2.54 s ± 64.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
%%timeit

# returns list of lists of dicts with data and metadata;
# the first list contains elements for each dependent parameter;
# second list contains all the independent parameters, 
# and the last element is the dependent parameter

data = get_data_by_id(datasaver.dataset.run_id)

5.84 s ± 322 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### with 'array' type

In [13]:
meas = Measurement()

meas.register_parameter(s1)
meas.register_parameter(s2)
meas.register_parameter(magn, setpoints=(s1, s2), paramtype='array')
meas.register_parameter(phas, setpoints=(s1, s2), paramtype='array')

In [14]:
meas.write_period = 10

In [15]:
# %%timeit

with meas.run() as datasaver:
    
    t0_dataset_array = time.perf_counter()
    
    for s1_val, s2_val, magn_vals, phas_vals in produce_measurement_data(n_pts_in_dim):
        
        datasaver.add_result((s1, s1_val), (s2, s2_val), (magn, magn_vals),
                             (phas, phas_vals))
    
t1_dataset_array = time.perf_counter()
print(f"Data saving to dataset with 'array' paramtype took {t1_dataset_array-t0_dataset_array} s")

dataset = datasaver.dataset

Starting experimental run with id: 3
Data saving to dataset with 'array' paramtype took 0.2915894999999864 s


In [16]:
%%timeit

# returns list of rows, as in sqlite (each row is a list where every item corresponds to the column)

data = dataset.get_data(*dataset.parameters.split(','))

291 ms ± 45.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
%%timeit

# returns list of lists of dicts with data and metadata;
# the first list contains elements for each dependent parameter;
# second list contains all the independent parameters, 
# and the last element is the dependent parameter
# data = get_data_by_id(datasaver.dataset.run_id)

data = get_data_by_id(dataset.run_id)

333 ms ± 5.46 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Via numpy npy file

In [18]:
# %%timeit

outfile = TemporaryFile()
outfile.close()

t0_npy = time.perf_counter()

results_np = np.zeros((4, n_pts_in_dim**4))
last_index = 0

for s1_val, s2_val, magn_vals, phas_vals in produce_measurement_data(n_pts_in_dim):
    
    n_pts = len(magn_vals)

    results_np[0, last_index:last_index+n_pts] = s1_val
    results_np[1, last_index:last_index+n_pts] = s2_val
    results_np[2, last_index:last_index+n_pts] = magn_vals
    results_np[3, last_index:last_index+n_pts] = phas_vals

    last_index += n_pts
            
np.save(outfile.name, results_np)

t1_npy = time.perf_counter()
print(f"Data saving to numpy npy file took {t1_npy-t0_npy} s")

Data saving to numpy npy file took 0.05394800000004807 s


In [19]:
%%timeit
data = np.load(outfile.name+'.npy')

5.21 ms ± 610 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Via numpy npy file and memmap

In [20]:
# %%timeit

outfile = TemporaryFile()
outfile.close()

t0_npy_mm = time.perf_counter()

results_mm = np.lib.format.open_memmap(
    outfile.name, mode='w+', shape=(4, n_pts_in_dim**4))

last_index = 0

for s1_val, s2_val, magn_vals, phas_vals in produce_measurement_data(n_pts_in_dim):
    
    n_pts = len(magn_vals)

    results_mm[0, last_index:last_index+n_pts] = s1_val
    results_mm[1, last_index:last_index+n_pts] = s2_val
    results_mm[2, last_index:last_index+n_pts] = magn_vals
    results_mm[3, last_index:last_index+n_pts] = phas_vals

    last_index += n_pts
            
del results_mm  # closes the file and performs final flushing

t1_npy_mm = time.perf_counter()
print(f"Data saving to numpy npy file via memory map took {t1_npy_mm-t0_npy_mm} s")

Data saving to numpy npy file via memory map took 0.06339409999998225 s


In [21]:
%%timeit
data = np.load(outfile.name)

5.04 ms ± 557 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Via hdf5 file (directly to the file)

In [22]:
# %%timeit

outfile = TemporaryFile()
outfile.close()

t0_hdf5_2 = time.perf_counter()

last_index = 0

with h5py.File(outfile.name, 'w') as f:
    ds = f.create_dataset('results', shape=(4, n_pts_in_dim**4))

    for s1_val, s2_val, magn_vals, phas_vals in produce_measurement_data(n_pts_in_dim):

        n_pts = len(magn_vals)

        ds[0, last_index:last_index+n_pts] = s1_val
        ds[1, last_index:last_index+n_pts] = s2_val
        ds[2, last_index:last_index+n_pts] = magn_vals
        ds[3, last_index:last_index+n_pts] = phas_vals

        last_index += n_pts    

t1_hdf5_2 = time.perf_counter()
print(f"Data saving to hdf5 file took {t1_hdf5_2-t0_hdf5_2} s")

Data saving to hdf5 file took 0.4450757999999837 s


In [23]:
%%timeit
with h5py.File(outfile.name, 'r') as f:
    data = np.array(f['results'], copy=True)

5.5 ms ± 725 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Via hdf5 file (from allocated numpy array)

In [25]:
# %%timeit

outfile = TemporaryFile()
outfile.close()

t0_hdf5 = time.perf_counter()

results_np = np.zeros((4, n_pts_in_dim**4))
last_index = 0

for s1_val, s2_val, magn_vals, phas_vals in produce_measurement_data(n_pts_in_dim):
            
    n_pts = len(magn_vals)

    results_np[0, last_index:last_index+n_pts] = s1_val
    results_np[1, last_index:last_index+n_pts] = s2_val
    results_np[2, last_index:last_index+n_pts] = magn_vals
    results_np[3, last_index:last_index+n_pts] = phas_vals

    last_index += n_pts
            
with h5py.File(outfile.name, 'w') as f:
    ds = f.create_dataset('results', data=results_np)

t1_hdf5 = time.perf_counter()
print(f"Data saving to hdf5 file took {t1_hdf5-t0_hdf5} s")

Data saving to hdf5 file took 0.05811900000003334 s
